In [50]:
import pandas as pd
import string
import re 
import nltk 
import pickle
from sklearn.preprocessing import LabelEncoder #to encode the features
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('stopwords') 
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
Dec = pd.read_csv("New test_7.csv" , encoding="ISO-8859-1") #read description csv
Dec.drop('Unnamed: 0',axis=1,inplace=True) #Drop Unnamed: 0 column
Dec.dropna(inplace=True) #drop null values

In [3]:
JawwyTV_df = pd.read_csv("Final_Dataset.txt",sep=",") #read the Final_Dataset csv
Stc_df = pd.read_excel("stc TV Data Set_T3.xlsb") #read the stc TV Data Set_T3 csv
JawwyTV_df.head(3)

,Unnamed: 0,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name
0,1,2017-05-27,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets
1,3,2017-05-21,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana
2,4,2017-08-10,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess


In [4]:
Stc_df.head(3)

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
0,0,26138,100 treets,1,2017-05-27,Drama
1,1,7946,Moana,1,2017-05-21,Animation
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation


In [5]:
JawwyTV_df.drop(columns="Unnamed: 0",inplace=True) #Drop Unnamed: 0 column

In [6]:
print(JawwyTV_df.shape)
print(Stc_df.shape)

(3598607, 12)
(1048575, 6)


In [7]:
JawwyTV_df.rename(columns={'original_name':'name'},inplace=True) #rename the columns

In [8]:
JawwyTV_df_new = JawwyTV_df.head(1048575) #limit the JawwyTV_df to become equal with Stc_df
JawwyTV_df_new['rating'] = Stc_df['rating'] #add rating column to JawwyTV_df

In [9]:
JawwyTV_df_new.info() #show the column name, null values,and Dtype 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   date_             1048575 non-null  object
 1   user_id_maped     1048575 non-null  int64 
 2   program_name      1048575 non-null  object
 3   duration_seconds  1048575 non-null  int64 
 4   program_class     1048575 non-null  object
 5   season            1048575 non-null  int64 
 6   episode           1048575 non-null  int64 
 7   program_desc      1034537 non-null  object
 8   program_genre     1048575 non-null  object
 9   series_title      1048575 non-null  int64 
 10  hd                1048575 non-null  int64 
 11  name              1048575 non-null  object
 12  rating            1048575 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 104.0+ MB


In [10]:
JawwyTV_df_new.describe(exclude='number') #generate descriptive statistics.

,date_,program_name,program_class,program_desc,program_genre,name
count,1048575,1048575,1048575,1034537,1048575,1048575
unique,411,8270,2,11176,16,1795
top,2017-04-21,The Boss Baby,SERIES/EPISODES,Animation MovieThe Boss Baby (HD),Animation,Friends
freq,4512,23625,560174,23625,401730,32997


In [11]:
JawwyTV_df_new.describe() #generate descriptive statistics.

,user_id_maped,duration_seconds,season,episode,series_title,hd,rating
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,1.709266e+04,1.230957e+03,1.342139e+00,6.157952e+00,1.205922e-02,3.862728e-01,2.497283e+00
std,1.003513e+04,6.821058e+03,2.104095e+00,1.222015e+01,1.091504e-01,4.868946e-01,1.119837e+00
min,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,8.253000e+03,5.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,1.714900e+04,1.190000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
75%,2.566500e+04,1.328000e+03,1.000000e+00,9.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00
max,3.428000e+04,1.461329e+06,2.300000e+01,2.820000e+02,1.000000e+00,1.000000e+00,4.000000e+00


In [12]:
JawwyTV_df_new.isnull().sum() #the sum of missing values

date_                   0
user_id_maped           0
program_name            0
duration_seconds        0
program_class           0
season                  0
episode                 0
program_desc        14038
program_genre           0
series_title            0
hd                      0
name                    0
rating                  0
dtype: int64

In [13]:
JawwyTV_df_new['program_name'].replace('Dunkirk ', 'Dunkirk',inplace=True) #remove space from the name

In [15]:
#set the program_genre as Action for Dunkirk 
JawwyTV_df_new.loc[JawwyTV_df_new['program_name'] == 'Dunkirk', 'program_genre' ] = 'Action' 
#set the program_genre as Animation for Batman Unlimited: Animal Instincts (for Viewing)
JawwyTV_df_new.loc[JawwyTV_df_new['program_name'] == 'Batman Unlimited: Animal Instincts (for Viewing)', 'program_genre' ] = 'Animation'

In [16]:
JawwyTV_df_new.loc[JawwyTV_df_new['name'] == 'Harry  ', 'program_genre' ] = 'Crime' #set the program_genre as Crime for Harry 
JawwyTV_df_new.loc[JawwyTV_df_new['name'] == 'Friends  ', 'program_genre' ] = 'Comedy' #set the program_genre as Comedy for Friends 
JawwyTV_df_new.loc[JawwyTV_df_new['name'] == 'Sehhee Wa aree  ', 'program_genre' ] = 'Comedy' #set the program_genre as Comedy for Sehhee Wa aree  

In [17]:
#reset the program_desc columns have the movies (genre,class,name,hd) 
for idx in JawwyTV_df_new.index:
     if JawwyTV_df_new.loc[idx,'hd']=="1":
             JawwyTV_df_new.loc[idx,'program_desc'] = JawwyTV_df_new.loc[idx,'program_genre'] + " "+ JawwyTV_df.loc[idx,'program_class']+ " "+JawwyTV_df.loc[idx,'name'] + " "+ '(HD)'
     else:
            JawwyTV_df_new.loc[idx,'program_desc'] = JawwyTV_df_new.loc[idx,'program_genre'] + " "+ JawwyTV_df.loc[idx,'program_class']+ " "+JawwyTV_df.loc[idx,'name']

In [226]:
JawwyTV_df_new.to_csv('JawwyTV_df_new.csv') #save the df to csv file

In [18]:
JawwyTV_df_new.drop(["season", "episode","series_title",'date_','program_name'],axis=1, inplace=True) #drop the colmns that we don't need

In [19]:
JawwyTV_df_new = JawwyTV_df_new.merge(Dec, how='left' , on='name') #join the description with movies names

In [20]:
JawwyTV_df_new['duration_hours'] = JawwyTV_df_new['duration_seconds'] / 3600 #add new column for duration hours by divide the sec to 3600
JawwyTV_df_new.drop("duration_seconds",axis=1, inplace=True) #drop the duration_seconds column

In [21]:
JawwyTV_df_new.drop('url',axis=1, inplace=True) #drop the url column after joining

In [22]:
JawwyTV_df_new.head(3)

,user_id_maped,program_class,program_desc,program_genre,hd,name,rating,dec,duration_hours
0,26138,MOVIE,Drama MOVIE 100 treets,Drama,0,100 treets,1,"Three people, three extraordinary stories. All...",0.011111
1,7946,MOVIE,Animation MOVIE Moana,Animation,1,Moana,1,"In Ancient Polynesia, when a terrible curse in...",0.004722
2,7418,MOVIE,Animation MOVIE The Mermaid Princess,Animation,1,The Mermaid Princess,1,"A kind-hearted mermaid, braving the Sea-God's ...",0.002222


In [23]:
JawwyTV_df_new.dropna(inplace=True) #drop null values

In [51]:
print(string.punctuation )

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [24]:
lem = WordNetLemmatizer() #init Lemmatizer
english_punctuations = string.punctuation #string constants for the punctuation
punctuations_list = english_punctuations + english_punctuations #create punctuations_list
def remove_punctuations(text): #remove punctuation from the text
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#text preprocessing function by lower the text, regular expression, stop word removel, and Lemmatizer
def processPost(text):
    text=text.lower() 
    for letter in '.(/ÀÈÌÒÙàèìòùÁÉÍÓÚÝáéíóúýÂÊÎÔÛâêîôûÃÑÕãñõÄËÏÖÜŸäëïöüÿ½ï':
        text = text.replace(letter,'')
    text = re.sub('@[^\s]+', ' ', text) 
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text) 
    text = re.sub(r'#([^\s]+)', r'\1', text)
    text= remove_punctuations(text)
    text=text.replace('read all', ' ')
    text_list = text.split()
    text_list =  [lem.lemmatize(word) for word in text_list if not word in set(stopwords.words('english')) and len(word) > 1 ]
    text = ' '.join(text_list)
    return text

In [25]:
Dec_df = JawwyTV_df_new[['dec', 'name','program_desc']].drop_duplicates() #init df with three columns

In [26]:
Dec_df['desc_new']=Dec_df['dec'].map(lambda x:processPost(x)) #apply preprocessing to description
Dec_df['program_desc_new']=Dec_df['program_desc'].map(lambda x:processPost(x)) #apply preprocessing to program description

In [30]:
Dec_df['full_Dec'] = Dec_df['desc_new'] + " " + Dec_df['program_desc_new'] #add the program description to the description

In [31]:
Dec_df.to_csv("Dec_df.csv") #save the description df

In [36]:
cv_Dec = CountVectorizer(max_df=0.80, min_df=2) #init Count Vectorizer to Convert a collection of text documents to a matrix of token counts.
Dec_matrix= cv_Dec.fit_transform(Dec_df['desc_new']) #fit and transform the description column
cv_ProgramDec = CountVectorizer(max_df=0.80, min_df=2) #init Count Vectorizer to Convert a collection of text documents to a matrix of token counts.
porgram_dec_matrix= cv_ProgramDec.fit_transform(Dec_df['program_desc_new']) #fit and transform the program description column
cv_full_Dec = CountVectorizer(max_df=0.80, min_df=2) #init Count Vectorizer to Convert a collection of text documents to a matrix of token counts.
full_Dec_matrix= cv_ProgramDec.fit_transform(Dec_df['full_Dec']) #fit and transform the full description column

In [37]:
pickle.dump(cv_Dec, open('cv_Dec.pkl', 'wb')) #save the Count Vectorizer
pickle.dump(Dec_matrix, open('Dec_matrix.pkl', 'wb')) #save the matrix
pickle.dump(cv_ProgramDec, open('cv_ProgramDec.pkl', 'wb')) #save the Count Vectorizer
pickle.dump(porgram_dec_matrix, open('porgram_dec_matrix.pkl', 'wb')) #save the matrix
pickle.dump(cv_full_Dec, open('cv_full_Dec.pkl', 'wb')) #save the Count Vectorizer
pickle.dump(full_Dec_matrix, open('full_Dec_matrix.pkl', 'wb')) #save the matrix

In [421]:
colf = JawwyTV_df_new[['user_id_maped','dec','name','rating']] #init df for Collaborative Filterting
colf['desc_new']=colf['dec'].map(lambda x:processPost(x)) #apply preprocessing to description

In [422]:
name_en = LabelEncoder() #init LabelEncoder
colf['name'] = name_en.fit_transform(colf['name']) #encode the name column

In [423]:
colf.to_csv('colf.csv') #save the df
pickle.dump(name_en, open('en.pkl', 'wb')) #save the Encoder